In [3]:
import ads
import json
import requests
import re
import io
import pandas as pd
import sys

In [4]:
sys.path.insert(0, '../')
from feature_class import Feature
from paper_class import Paper

In [2]:
# routine to read information from json file
# creates a Feature instance with extracted data
def feature_extract(obj):
        if '__type__' in obj and obj['__type__'] == 'Feature':
            return Feature(obj['name'], obj['id'], obj['polygon_coordinates'], obj['publications'])
        return Feature("None", 0, "None", "None")

In [3]:
# routine to parse json file and fetch definition of each feature
def parse_json_stream(text):
    list_idx = [m.start() for m in re.finditer('}\n', text)]
    list_objs = []
    last_index = 0
    for index in list_idx:
        new_feature = json.loads(text[last_index:index+2], object_hook=feature_extract)
        if (new_feature.name == "None"):
            break
        list_objs += [new_feature]
        last_index = index+2
    return list_objs

In [4]:
json_name = "features.json"
features = []

In [5]:
with open(json_name) as fin:
    features = parse_json_stream(fin.read())

In [6]:
# write a json file with extended list of publications for each feature
# publications are requested from ADS API
# list of features is stored in features variable
with io.open('features_extended.json', 'w', encoding='utf-8') as fout:
    for feature in features:
        print(feature.name)
        # request ADS API for publications based on feature name
        # store objects of type Article
        papers = []
        try:
            papers = list(ads.SearchQuery(q=feature.name, fl=['title', 'author', 'year', 'pub', 'bibcode']))
        except (ads.exceptions.APIResponseError, ads.exceptions.SolrResponseParseError) as e:
            "Error: {}".format(e)
            continue
        
        citation_str = []
        # extract several fields from each publication
        # whole list of fields can be found in https://github.com/adsabs/adsabs-dev-api/blob/master/search.md#fields
        for paper in papers:
            p = Paper(paper.title[0], paper.author, paper.year, paper.pub, paper.bibcode)
            citation_str.append(p)
        # extend feature's list of publications 
        feature.addPublications(citation_str)
        #write extended feature info to a new json file
        fout.write(str(json.dumps(feature.dump(), ensure_ascii=False, indent=4))+"\n")
fout.close()

Abalos Colles
Abalos Mensa
Abalos Scopuli
Abalos Undae
Aban
Abus Vallis
Achar


In [7]:
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame([feature.to_dict() for feature in features])

In [8]:
#print(df["publications"][0])

In [3]:
test = list(ads.SearchQuery(q="Ajon", fl=['title', 'author', 'year', 'pub', 'bibcode']))

In [5]:
for t in test:
    print(t.title[0])
    print(t.author)
    print(t.year)
    print(t.pub)
    print(t.bibcode)

Theoretical Treatment of Spectrophotometric Tyrosine Titration of Two β-Lactamases of Class A
['Miteva, M.', 'Shosheva, Al.', 'Sose, Al.', 'Makinen, M.', 'Atanasov, B.']
2001
Comptes Rendus de l'Academie Bulgare des Sciences
2001CRABS..54d..93M
27-DAY Varations in the IMF Components and the Cosmic Ray Anisotropies
['Tolba, M. F.', 'Pomerantz, M. A.', 'Duggal, S. P.', 'Owens, A. J.']
1981
International Cosmic Ray Conference
1981ICRC....4...52T
Instructions for Authors Contributing to WGN
['Arlt, R.']
2002
WGN, Journal of the International Meteor Organization
2002JIMO...30...83A
The Aurora - 1983
['Livesey, R. J.']
1985
Journal of the British Astronomical Association
1985JBAA...95..100L
Domain wall motion influence on the magnetoimpedance effect
['Gutiérrez, J.', 'Atkinson, D.', 'Squire, P. T.', 'Barandiarán, J. M.']
1999
Journal of Magnetism and Magnetic Materials
1999JMMM..196..169G
Nonlinear excess conductivity in superconducting aluminum films above the transition temperature
['Kajim

TypeError: 'NoneType' object is not subscriptable